In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns; sns.set()  # for plot styling


from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler, minmax_scale

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D # for 3D plots

from sklearn.cluster import MeanShift, estimate_bandwidth

from itertools import cycle
from sklearn.mixture import GaussianMixture

import statsmodels.api as sm

import statsmodels.formula.api as smf

# Composicion de pipelines
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
# Regresion lineal
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Importar/ Exportar modelos
from joblib import dump, load

# Metricas
from sklearn.metrics import mean_squared_error as mse

# q-q plots
import scipy.stats as stats

import statsmodels.api as sm

import statsmodels.formula.api as smf

from pandas_profiling import ProfileReport

# Para preparar los datos
from sklearn.preprocessing import LabelEncoder

# Para crear el arbol de decisión 
from sklearn.tree import DecisionTreeClassifier 

# Para usar KNN como clasificador
from sklearn.neighbors import KNeighborsClassifier

# Para usar Random Tree Forest como clasificador
from sklearn.ensemble import RandomForestClassifier

# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import model_selection

# Para evaluar el modelo
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix

# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV

# Para la validación cruzada
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

#Librerías para la visualización
import matplotlib.pyplot as plt

# Seaborn
import seaborn as sns 
from sklearn import tree
from sklearn.preprocessing import LabelEncoder

#SMF
import statsmodels.formula.api as smf

from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR

# check version number
import imblearn

from collections import Counter

from imblearn.over_sampling import SMOTE, ADASYN

In [2]:
# Se cargan los datos
# Ubicación de los datos, deben estar en la misma carpeta que este notebook
pais = "brasil"
liga = "brasileirao-serie-a"
version = 0
db_location = f"data/{pais}_{liga}/version{version}/resultados_anteriores.xlsx"
db_location2 = f"data/{pais}_{liga}/version{version}/proximos_partidos.xlsx"

# db_location= "data_matches_mismarcadores_half0.xlsx"


# Se leen los datos, estan separados con ','
# data1 = pd.read_excel(db_location, sheet_name="Premier League 20222023")
# data2 = pd.read_excel(db_location, sheet_name="Premier League 20212022")
# data = pd.concat([data1,data2])
data = pd.read_excel(db_location)

proximos_partidos= pd.read_excel(db_location2)

In [ ]:
# data.dropna(subset=['HxG','AxG'],inplace=True)

In [3]:
data.fillna(value=0,inplace=True)
data

,Date,HomeTeam,AwayTeam,HG,AG,HxG,AxG,HP,AP,HTS,...,ATP,HPC,APC,HT,AT,HA,AA,HDA,ADA,Resultado
0,06.08.2023 18:00,Cuiabá,Flamengo,3,0,2.15,0.76,42%,58%,16,...,539.0,264.0,474.0,0.0,0.0,70,122,46,87,3
1,06.08.2023 16:30,Bahia,América-MG,3,1,1.49,0.62,61%,39%,23,...,260.0,417.0,187.0,0.0,0.0,119,73,62,20,2
2,06.08.2023 16:30,Coritiba,Bragantino,0,1,0.40,1.07,50%,50%,9,...,357.0,300.0,261.0,0.0,0.0,89,102,53,48,-1
3,06.08.2023 16:30,Cruzeiro,Botafogo,0,0,1.06,0.07,59%,41%,11,...,270.0,515.0,203.0,0.0,0.0,153,61,54,12,0
4,06.08.2023 14:00,Sao Paulo,Atlético-MG,0,2,0.94,2.06,62%,38%,16,...,290.0,546.0,220.0,0.0,0.0,152,65,79,23,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,15.04.2023 16:30,Botafogo,Sao Paulo,2,1,0.00,0.00,32%,68%,12,...,646.0,229.0,574.0,10.0,16.0,85,122,57,41,1
174,15.04.2023 16:30,Bragantino,Bahia,2,1,0.00,0.00,56%,44%,13,...,343.0,339.0,248.0,26.0,11.0,127,87,54,39,1
175,15.04.2023 16:30,Fortaleza,Internacional,1,1,0.00,0.00,47%,53%,24,...,474.0,319.0,379.0,17.0,19.0,126,110,73,30,0
176,15.04.2023 14:00,América-MG,Fluminense,0,3,1.14,3.26,46%,54%,17,...,442.0,267.0,364.0,18.0,14.0,114,56,95,22,-3


In [4]:
def conseguir_anio(texto):
    return (texto.split()[0]).split(".")[2]
def conseguir_mes(texto):
    return (texto.split()[0]).split(".")[1]
def conseguir_dia(texto):
    return (texto.split()[0]).split(".")[0]
def quitar_por(texto):
    return (texto.replace("%",""))    

In [5]:
data['year'] = (data['Date'].apply(conseguir_anio))
data['month'] = (data['Date'].apply(conseguir_mes))
data['day'] = (data['Date'].apply(conseguir_dia))
data['HP'] = data['HP'].apply(quitar_por)
data['AP'] = data['AP'].apply(quitar_por)
data['year'] = data['year'].astype(int)
data['month'] = data['month'].astype(int)
data['day'] = data['day'].astype(int)
data['HP'] = data['HP'].astype(int)
data['AP'] = data['AP'].astype(int)
data['Fecha']= pd.to_datetime(data[['year','month','day']])

data['TG'] = data['HG'] + data['AG']
# data['Mas2Goles'] = np.where(data['TG'] > 2,1,0)

data_backup= data.copy()

In [6]:
data_backup['Resultado_y'] = np.where(data_backup['Resultado']>=1,1,np.where(data_backup['Resultado']==0,0,-1))

In [7]:
data_backup['TG'].value_counts()

2    50
3    42
1    32
4    21
0    16
5    11
6     5
7     1
Name: TG, dtype: int64

In [8]:
data_backup_2 = data_backup[(data_backup['TG']>0) & (data_backup['TG']<6)].copy()

In [9]:
data_backup_2['TG'].value_counts()

2    50
3    42
1    32
4    21
5    11
Name: TG, dtype: int64

In [10]:
selected_cols = ['HTS', 'ATS', 'HSI', 'ASI', 'HSO', 'ASO', 'HBS', 'ABS', 'HGS', 'AGS']

# Proceso de datos Goles

In [11]:
x_drop=data_backup_2[selected_cols]
feature_names= selected_cols
X_gol = np.array(x_drop)
y_gol = np.array(data_backup_2['TG'])

In [12]:
oversample_gol = SMOTE()
X_gol, y_gol = oversample_gol.fit_resample(X_gol, y_gol)

In [13]:
ss_gol = StandardScaler()
X_gol = ss_gol.fit_transform(X_gol)

# Proceso de datos Resultado_y

## Selección sub-datasets

In [14]:
x_drop=data_backup[selected_cols]
feature_names= selected_cols
X_resul = np.array(x_drop)
y_resul = np.array(data_backup['Resultado_y'])

## SMOTE

In [15]:
oversample_resul = SMOTE()
X_resul, y_resul = oversample_resul.fit_resample(X_resul, y_resul)

## Estandarización

In [16]:
ss_resul = StandardScaler()
X_resul = ss_resul.fit_transform(X_resul)

# Predicción Próximos Partidos

In [17]:
X_predicc= X_resul.copy()
y_predicc= y_resul.copy()

model_resul=LogisticRegression(random_state=1001)

model_resul.fit(X_predicc, y_predicc)

LogisticRegression(random_state=1001)

In [18]:
X_predicc_gol= X_gol.copy()
y_predicc_gol= y_gol.copy()

model_gol=LinearDiscriminantAnalysis()

model_gol.fit(X_predicc_gol, y_predicc_gol)

LinearDiscriminantAnalysis()

In [19]:
proximos_partidos = proximos_partidos[['Date','HomeTeam','AwayTeam']]

proximos_partidos['year'] = (proximos_partidos['Date'].apply(conseguir_anio))
proximos_partidos['month'] = (proximos_partidos['Date'].apply(conseguir_mes))
proximos_partidos['day'] = (proximos_partidos['Date'].apply(conseguir_dia))
proximos_partidos['year'] = proximos_partidos['year'].astype(int)
proximos_partidos['month'] = proximos_partidos['month'].astype(int)
proximos_partidos['day'] = proximos_partidos['day'].astype(int)
proximos_partidos['Fecha']= pd.to_datetime(proximos_partidos[['year','month','day']])

In [20]:
data.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'HG', 'AG', 'HxG', 'AxG', 'HP', 'AP',
       'HTS', 'ATS', 'HSI', 'ASI', 'HSO', 'ASO', 'HBS', 'ABS', 'HFK', 'AFK',
       'HC', 'AC', 'HOFF', 'AOFF', 'HTI', 'ATI', 'HGS', 'AGS', 'HF', 'AF',
       'HRC', 'ARC', 'HYC', 'AYC', 'HTP', 'ATP', 'HPC', 'APC', 'HT', 'AT',
       'HA', 'AA', 'HDA', 'ADA', 'Resultado', 'year', 'month', 'day', 'Fecha',
       'TG'],
      dtype='object')

In [40]:
(data[(data['HomeTeam'] == "Botafogo")]).copy().drop(['Resultado', 'year','month', 'day','Fecha','TG'],axis=1).head(5).mean()

HG       2.400
AG       0.200
HxG      1.832
AxG      0.744
HP      52.400
        ...   
AT       0.000
HA     112.200
AA     112.400
HDA     48.200
ADA     34.400
Length: 40, dtype: float64

In [44]:
def calcularDiferenciaGol5Prom_pred():
    dataNueva= []
    for i in range(len(proximos_partidos)):
        infoAct=[]
        homeTeam = proximos_partidos.iloc[i]['HomeTeam']
        awayTeam = proximos_partidos.iloc[i]['AwayTeam']
        fecha_match =  proximos_partidos.iloc[i]['Fecha']
        infoHome = data[(data['HomeTeam'] == homeTeam) &  (data['Fecha'] < fecha_match )].copy().head(5)
        infoAway = data[(data['AwayTeam'] == awayTeam) & (data['Fecha'] < fecha_match )].copy().head(5)
        infoHome.drop(['Resultado', 'year','month', 'day','Fecha','TG'], axis=1,inplace=True)
        infoAway.drop(['Resultado', 'year','month', 'day','Fecha','TG'], axis=1,inplace=True)
        ##Solo se va a tomar en cuenta la información de cada equipo , más no las estadisticas que recibe en contra.
        infoHome = infoHome.mean()
        infoAway = infoAway.mean()
        if len(infoHome) > 40:
            for i in range(len(infoHome)):
                infoAct.append(0)
            dataNueva.append(infoAct)
            continue
        for i in range(0,len(infoHome),2):
            infoAct.append(infoHome[i])
            infoAct.append(infoAway[i+1])
        dataNueva.append(infoAct)
    return dataNueva

In [45]:
datosProm5Partidos_prox = calcularDiferenciaGol5Prom_pred()
cols= ["Prom5_HG","Prom5_AG","Prom5_HXG","Prom5_AXG","Prom5_HP","Prom5_AP","Prom5_HTS","Prom5_ATS","Prom5_HSI","Prom5_ASI","Prom5_HSO","Prom5_ASO","Prom5_HBS","Prom5_ABS","Prom5_HFK","Prom5_AFK","Prom5_HC","Prom5_AC","Prom5_HOF","Prom5_AOF","Prom5_HTI","Prom5_ATI","Prom5_HGS","Prom5_AGS","Prom5_HF","Prom5_AF","Prom5_HRC","Prom5_ARC","Prom5_HYC","Prom5_AYC","Prom5_HTP","Prom5_ATP","Prom5_HPC","Prom5_APC","Prom5_HT","Prom5_AT","Prom5_HA","Prom5_AA","Prom5_HDA","Prom5_ADA"]


In [46]:
dataProm5P_prox= pd.DataFrame(np.array(datosProm5Partidos_prox), columns=cols)
dataProm5P_prox

,Prom5_HG,Prom5_AG,Prom5_HXG,Prom5_AXG,Prom5_HP,Prom5_AP,Prom5_HTS,Prom5_ATS,Prom5_HSI,Prom5_ASI,...,Prom5_HTP,Prom5_ATP,Prom5_HPC,Prom5_APC,Prom5_HT,Prom5_AT,Prom5_HA,Prom5_AA,Prom5_HDA,Prom5_ADA
0,2.4,0.8,1.832,0.816,52.4,49.6,15.2,7.8,6.2,2.6,...,466.2,417.4,391.4,337.4,0.0,0.0,112.2,100.6,48.2,36.6
1,1.0,0.4,1.270,0.460,50.0,49.0,16.4,9.0,5.4,3.2,...,471.2,367.2,386.8,304.4,0.0,2.6,107.0,98.4,39.8,35.4
2,1.6,1.2,1.264,0.656,41.6,44.6,13.0,6.8,6.2,2.2,...,330.2,366.6,251.4,296.4,0.0,0.0,103.4,92.0,52.0,34.8
3,1.4,1.0,1.054,1.012,44.0,46.8,13.2,12.0,5.0,6.0,...,463.8,373.2,385.4,303.8,0.0,0.0,94.4,100.4,47.0,36.2
4,2.2,0.4,2.030,0.846,47.6,60.8,17.2,10.6,6.0,3.2,...,455.4,553.2,366.6,485.8,0.0,0.0,119.4,137.4,61.0,61.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1.0,1.2,1.574,1.098,45.4,44.2,15.6,13.6,5.2,4.0,...,386.6,348.8,311.0,267.4,0.0,2.6,106.0,97.0,51.2,39.4
196,1.0,1.2,1.240,0.704,53.4,48.8,12.8,9.0,3.6,3.8,...,534.6,405.4,459.8,327.2,0.0,0.0,111.6,96.0,57.6,30.0
197,1.8,0.4,1.172,0.888,45.6,47.2,10.0,11.2,4.0,4.2,...,392.2,383.2,318.8,306.0,0.0,0.0,90.6,96.0,40.2,37.2
198,1.4,1.2,1.622,0.978,57.4,57.0,16.4,8.6,5.6,4.0,...,414.4,545.2,357.4,477.6,0.0,0.0,130.8,117.0,57.6,61.0


In [47]:
proximos_partidos= proximos_partidos.reset_index(drop=True)
dataProm5P_prox = dataProm5P_prox.reset_index(drop=True)

In [48]:
proximos_partidos

,Date,HomeTeam,AwayTeam,year,month,day,Fecha
0,12.08.2023 19:00,Botafogo,Internacional,2023,8,12,2023-08-12
1,13.08.2023 09:00,Atlético-MG,Bahia,2023,8,13,2023-08-13
2,13.08.2023 14:00,América-MG,Goiás,2023,8,13,2023-08-13
3,13.08.2023 14:00,Corinthians,Coritiba,2023,8,13,2023-08-13
4,13.08.2023 14:00,Gremio,Fluminense,2023,8,13,2023-08-13
...,...,...,...,...,...,...,...
195,03.12.2023 16:00,Goiás,América-MG,2023,12,3,2023-12-03
196,03.12.2023 16:00,Internacional,Botafogo,2023,12,3,2023-12-03
197,03.12.2023 16:00,Santos,Fortaleza,2023,12,3,2023-12-03
198,03.12.2023 16:00,Sao Paulo,Flamengo,2023,12,3,2023-12-03


In [49]:
dataProm5P_mod_prox= pd.concat([proximos_partidos, dataProm5P_prox], axis=1)

In [50]:
dataProm5P_mod_prox

,Date,HomeTeam,AwayTeam,year,month,day,Fecha,Prom5_HG,Prom5_AG,Prom5_HXG,...,Prom5_HTP,Prom5_ATP,Prom5_HPC,Prom5_APC,Prom5_HT,Prom5_AT,Prom5_HA,Prom5_AA,Prom5_HDA,Prom5_ADA
0,12.08.2023 19:00,Botafogo,Internacional,2023,8,12,2023-08-12,2.4,0.8,1.832,...,466.2,417.4,391.4,337.4,0.0,0.0,112.2,100.6,48.2,36.6
1,13.08.2023 09:00,Atlético-MG,Bahia,2023,8,13,2023-08-13,1.0,0.4,1.270,...,471.2,367.2,386.8,304.4,0.0,2.6,107.0,98.4,39.8,35.4
2,13.08.2023 14:00,América-MG,Goiás,2023,8,13,2023-08-13,1.6,1.2,1.264,...,330.2,366.6,251.4,296.4,0.0,0.0,103.4,92.0,52.0,34.8
3,13.08.2023 14:00,Corinthians,Coritiba,2023,8,13,2023-08-13,1.4,1.0,1.054,...,463.8,373.2,385.4,303.8,0.0,0.0,94.4,100.4,47.0,36.2
4,13.08.2023 14:00,Gremio,Fluminense,2023,8,13,2023-08-13,2.2,0.4,2.030,...,455.4,553.2,366.6,485.8,0.0,0.0,119.4,137.4,61.0,61.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,03.12.2023 16:00,Goiás,América-MG,2023,12,3,2023-12-03,1.0,1.2,1.574,...,386.6,348.8,311.0,267.4,0.0,2.6,106.0,97.0,51.2,39.4
196,03.12.2023 16:00,Internacional,Botafogo,2023,12,3,2023-12-03,1.0,1.2,1.240,...,534.6,405.4,459.8,327.2,0.0,0.0,111.6,96.0,57.6,30.0
197,03.12.2023 16:00,Santos,Fortaleza,2023,12,3,2023-12-03,1.8,0.4,1.172,...,392.2,383.2,318.8,306.0,0.0,0.0,90.6,96.0,40.2,37.2
198,03.12.2023 16:00,Sao Paulo,Flamengo,2023,12,3,2023-12-03,1.4,1.2,1.622,...,414.4,545.2,357.4,477.6,0.0,0.0,130.8,117.0,57.6,61.0


In [51]:
def generar_predicciones_pred():
    predicciones_partido=[]
    predicciones_gol=[]
    proba_local=[]
    proba_empate=[]
    proba_visitante=[]
    for i in range(len(dataProm5P_mod_prox)):
        x_int= [dataProm5P_mod_prox.iloc[i]['Prom5_HTS'],dataProm5P_mod_prox.iloc[i]['Prom5_ATS'],
        dataProm5P_mod_prox.iloc[i]['Prom5_HSI'],dataProm5P_mod_prox.iloc[i]['Prom5_ASI'],dataProm5P_mod_prox.iloc[i]['Prom5_HSO'],dataProm5P_mod_prox.iloc[i]['Prom5_ASO'],dataProm5P_mod_prox.iloc[i]['Prom5_HBS'],
        dataProm5P_mod_prox.iloc[i]['Prom5_ABS'],dataProm5P_mod_prox.iloc[i]['Prom5_HGS'], dataProm5P_mod_prox.iloc[i]['Prom5_AGS'] ]
        x_int= np.array(x_int)
        # Estandarizacion 
        x_int_estd = ss_resul.transform([x_int])
        x_int_estd_gol = ss_gol.transform([x_int])
        feature_names_act= ['HTS', 'ATS', 'HSI', 'ASI', 'HSO', 'ASO', 'HBS', 'ABS', 'HGS', 'AGS']

        x_int_estd = pd.DataFrame(x_int_estd,columns=feature_names_act)
        x_int_estd= np.array(x_int_estd)

        x_int_estd_gol = pd.DataFrame(x_int_estd_gol,columns=feature_names_act)
        x_int_estd_gol= np.array(x_int_estd_gol)

        predicc = model_resul.predict(x_int_estd)
        predicc_gol = model_gol.predict(x_int_estd_gol)

        predicc_proba= model_resul.predict_proba(x_int_estd)
        proba_local.append((predicc_proba[0])[2])
        proba_empate.append((predicc_proba[0])[1])
        proba_visitante.append((predicc_proba[0])[0])
        predicciones_partido.append(predicc)
        predicciones_gol.append(predicc_gol)
    return [predicciones_partido,predicciones_gol,proba_local,proba_empate,proba_visitante]

In [52]:
resultados= generar_predicciones_pred()
predicciones_partido= resultados[0]
predicciones_gol = resultados[1]
proba_local= resultados[2]
proba_empate= resultados[3]
proba_visitante= resultados[4]
dataProm5P_mod_prox['Resultado_predicc']= pd.Series(predicciones_partido)
dataProm5P_mod_prox['Resultado_predicc_mod']= np.where(dataProm5P_mod_prox['Resultado_predicc']==1,"Local",np.where(dataProm5P_mod_prox['Resultado_predicc']==0,"Empate","Visitante"))
dataProm5P_mod_prox['Prediccion_gol']= pd.Series(predicciones_gol)
dataProm5P_mod_prox['Prediccion_gol_apuesta'] = np.where(((dataProm5P_mod_prox['Prediccion_gol']>=3)),"Hándicap +2 Goles o +1.5 Goles","Hándicap -4 Goles o -3.5 Goles")
dataProm5P_mod_prox['proba_local']= pd.Series(proba_local)
dataProm5P_mod_prox['proba_empate']= pd.Series(proba_empate)
dataProm5P_mod_prox['proba_visitante']= pd.Series(proba_visitante)

In [ ]:
# dataProm5P_mod_prox['proba_local']= 

In [58]:
pd.options.display.max_rows = 20

In [64]:
 dataProm5P_mod_prox[['Date','HomeTeam','AwayTeam','Prediccion_gol_apuesta','Prediccion_gol','Resultado_predicc_mod','proba_local','proba_empate','proba_visitante']].head(20)

,Date,HomeTeam,AwayTeam,Prediccion_gol_apuesta,Prediccion_gol,Resultado_predicc_mod,proba_local,proba_empate,proba_visitante
0,12.08.2023 19:00,Botafogo,Internacional,Hándicap -4 Goles o -3.5 Goles,[1],Local,0.990347,0.009637,0.000016
1,13.08.2023 09:00,Atlético-MG,Bahia,Hándicap -4 Goles o -3.5 Goles,[1],Visitante,0.006678,0.262729,0.730593
2,13.08.2023 14:00,América-MG,Goiás,Hándicap +2 Goles o +1.5 Goles,[3],Local,0.968585,0.031146,0.000269
3,13.08.2023 14:00,Corinthians,Coritiba,Hándicap +2 Goles o +1.5 Goles,[4],Visitante,0.002794,0.082885,0.914322
4,13.08.2023 14:00,Gremio,Fluminense,Hándicap -4 Goles o -3.5 Goles,[2],Local,0.860221,0.136291,0.003489
5,13.08.2023 16:30,Flamengo,Sao Paulo,Hándicap -4 Goles o -3.5 Goles,[1],Local,0.942401,0.057426,0.000174
6,13.08.2023 16:30,Fortaleza,Santos,Hándicap -4 Goles o -3.5 Goles,[1],Empate,0.082607,0.758785,0.158608
7,14.08.2023 17:00,Palmeiras,Cruzeiro,Hándicap +2 Goles o +1.5 Goles,[5],Local,0.628214,0.266087,0.105699
8,14.08.2023 19:00,Bragantino,Vasco,Hándicap +2 Goles o +1.5 Goles,[5],Local,0.731689,0.214421,0.053891
9,15.08.2023 18:00,Athletico-PR,Cuiabá,Hándicap +2 Goles o +1.5 Goles,[3],Local,0.913428,0.083918,0.002654


In [60]:
pd.options.display.max_rows = 20


# TODO: PROBAR CON PROMEDIO DE 10 PARTIDOS MÁXIMO Y CON PROMEDIO SIN DISCRMINAR LOCALIA O VISTA
#TODO: TUNEAR LOS MODELOS

# <a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ed14a935-a2d1-49c6-bc5b-8778463ac475' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>